<a href="https://colab.research.google.com/github/HarshaVardhanLanka/OpenDeepResearchAgent/blob/main/Day_7(Model_2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U google-generativeai tavily-python pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.5/329.5 kB 6.2 MB/s eta 0:00:00


In [2]:
import os
import io
import json
import datetime
import google.generativeai as genai
from tavily import TavilyClient
from google.colab import userdata
from google.colab import files
import pypdf
from IPython.display import display, Markdown

# --- API CONFIGURATION ---
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    TAVILY_API_KEY = userdata.get('TAVILY')
except:
    print("⚠️ Error: Please set GOOGLE_API_KEY and TAVILY_API_KEY in Colab Secrets.")

genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-pro-latest')
tavily = TavilyClient(api_key=TAVILY_API_KEY)
print("Setup complete.")

Setup complete.


In [3]:
# --- MEMORY SYSTEM ---
class HistoryManager:
    def __init__(self):
        self.history_file = "agent_history.json"
        self.history = self.load_history()

    def load_history(self):
        """Loads history from a JSON file if it exists."""
        if os.path.exists(self.history_file):
            with open(self.history_file, 'r') as f:
                return json.load(f)
        return []

    def save_entry(self, input_text, mode, final_report):
        """Saves a new research session to memory."""
        entry = {
            "id": len(self.history) + 1,
            "timestamp": str(datetime.datetime.now().strftime("%Y-%m-%d %H:%M")),
            "mode": mode,
            "input": input_text[:15] + "..." if len(input_text) > 50 else input_text, # Preview only
            "full_input": input_text,
            "report": final_report
        }
        self.history.append(entry)
        with open(self.history_file, 'w') as f:
            json.dump(self.history, f)
        print("✅ Research saved to History.")

    def show_history(self):
        """Displays list of past inputs."""
        if not self.history:
            print("\n📭 History is empty.")
            return None

        print("\n📚 RESEARCH HISTORY:")
        print(f"{'ID':<4} | {'Time':<18} | {'Mode':<8} | {'Topic/Input'}")
        print("-" * 60)
        for item in self.history:
            print(f"{item['id']:<4} | {item['timestamp']:<18} | {item['mode']:<8} | {item['input']}")
        return self.history
# Initialize Memory
memory = HistoryManager()

# --- HELPER: PDF TEXT EXTRACTOR ---
def extract_pdf_text(uploaded_file_bytes):
    try:
        pdf_reader = pypdf.PdfReader(io.BytesIO(uploaded_file_bytes))
        text = ""
        for page in pdf_reader.pages:
            text += page.extract_text() + "\n"
        return text
    except Exception as e:
        return f"Error reading PDF: {e}"

# --- STRICT AGENT STEPS ---

def step_1_generate_questions(input_text, mode):
    """Generates 3 questions based on mode."""
    print(f"\nStep 1: Generating 3 sub-questions (Mode: {mode})...")

    if mode == "topic":
        prompt = f"""
        Task: Break down the topic "{input_text}" into exactly 3 searchable questions.
        Constraint: Output ONLY the 3 questions separated by newlines. No numbering.
        """
    else: # mode == pdf
        prompt = f"""
        Task: Read the following paper excerpt and generate 3 search questions to VALIDATE the claims or check for newer updates.
        Paper Context: {input_text[:10000]}
        Constraint: Output ONLY the 3 questions separated by newlines. No numbering.
        """

    response = model.generate_content(prompt)
    questions = [q.strip() for q in response.text.strip().split('\n') if q.strip()]
    return questions[:3]

def step_2_get_responses(questions):
    """Retrieves short responses (Max 1 result per question)."""
    print(f"Step 2: Retrieving short responses from the web...")
    q_and_a_results = []

    for i, q in enumerate(questions, 1):
        print(f"   {i}. Searching: {q}")
        try:
            # max_results=1 forces a short response
            search_result = tavily.search(query=q, search_depth="basic", max_results=1)

            if search_result['results']:
                content = search_result['results'][0]['content'][:600] # Limit char count
                q_and_a_results.append(f"Question: {q}\nFound Info: {content}")
            else:
                q_and_a_results.append(f"Question: {q}\nFound Info: No data.")
        except:
             q_and_a_results.append(f"Question: {q}\nFound Info: Search Failed.")

    return "\n---\n".join(q_and_a_results)

def step_3_summarize(input_text, research_data, mode):
    prompt = f"Summarize user input '{input_text[:5000]}' combined with search data '{research_data}' into a Markdown report."
    response = model.generate_content(prompt)
    return response.text

In [4]:
# --- MAIN LOOP ---

def main():
    while True:
        print("\n" + "="*15 + " STRICT HYBRID AGENT " + "="*15)
        print("1. Research a Topic (Text 💬)")
        print("2. Research a Paper (Upload PDF 📖)")
        print("3. View History 📜")
        print("4. Exit 📤")

        choice = input("Select Option: ")

        if choice == '4':
            print("Goodbye.👋")
            break

        # --- OPTION 3: HISTORY ---
        if choice == '3':
            history_data = memory.show_history()
            if history_data:
                sub_choice = input("\nEnter ID to view full report (or Press Enter to go back): ")
                if sub_choice.isdigit():
                    idx = int(sub_choice) - 1
                    if 0 <= idx < len(history_data):
                        entry = history_data[idx]
                        print(f"\n🔁 RECALLING REPORT FOR: {entry['input']}")
                        print("-" * 50)
                        display(Markdown(entry['report']))
                        input("\nPress Enter to continue...")
            continue # Skip the rest and go back to menu

        # SETUP INPUT
        mode = ""
        input_data = ""
        label = "" # Used for history title

        if choice == '1':
            mode = "topic"
            input_data = input("\nEnter Research Topic: ")
            label = input_data

        elif choice == '2':
            mode = "pdf"
            print("\n⬆️ Upload PDF now:")
            uploaded = files.upload()
            if uploaded:
                filename = next(iter(uploaded))
                input_data = extract_pdf_text(uploaded[filename])
                label = f"PDF: {filename}"
            else:
                print("No file uploaded.")
                continue

        if input_data:
            # --- EXECUTE 3-STEP PROCESS ---

            # 1. Generate Questions
            questions = step_1_generate_questions(input_data, mode)
            print(f"   -> Plan: {questions}")

            # 2. Retrieve Responses
            raw_data = step_2_get_responses(questions)

            # 3. Summarize
            final_report = step_3_summarize(input_data, raw_data, mode)

            print("\n" + "="*20 + " 🎯 FINAL REPORT: " + "="*20 + "\n")
            display(Markdown(final_report))

            # Save to History
            memory.save_entry(label, mode, final_report)

if __name__ == "__main__":
    main()


=============== STRICT HYBRID AGENT ===============
1. Research a Topic (Text 💬)
2. Research a Paper (Upload PDF 📖)
3. View History 📜
4. Exit 📤
Select Option: 3

📭 History is empty.

=============== STRICT HYBRID AGENT ===============
1. Research a Topic (Text 💬)
2. Research a Paper (Upload PDF 📖)
3. View History 📜
4. Exit 📤
Select Option: 1

Enter Research Topic: Ocular Toxoplasmosis

Step 1: Generating 3 sub-questions (Mode: topic)...
   -> Plan: ['What are the causes and symptoms of ocular toxoplasmosis?', 'How is ocular toxoplasmosis diagnosed and treated?', 'What are the long-term complications and prevention strategies for ocular toxoplasmosis?']
Step 2: Retrieving short responses from the web...
   1. Searching: What are the causes and symptoms of ocular toxoplasmosis?
   2. Searching: How is ocular toxoplasmosis diagnosed and treated?
   3. Searching: What are the long-term complications and prevention strategies for ocular toxoplasmosis?

==================== 🎯 FINAL REPORT: 

# Ocular Toxoplasmosis

## Causes
Ocular toxoplasmosis is a disease caused by an infection with the parasite *Toxoplasma gondii*. The infection can be acquired congenitally (at birth) or postnatally, with postnatal infection now considered the more common cause.

## Symptoms and Clinical Features
The diagnosis is primarily made through clinical observation of characteristic eye lesions. Key signs and symptoms include:
*   Blurred vision
*   Focal necrotizing retinochoroiditis (inflammation and tissue death in the retina and choroid)
*   Retinal scars from past inflammation
*   Segmental retinal arteritis with Kyrieleis plaques (in atypical cases)

## Diagnosis
Diagnosis is mainly based on a clinical eye examination that reveals a focal necrotizing retinochoroiditis. It is important to note that an immunosuppressed patient may have an active ocular toxoplasmosis infection even with completely negative immunoglobulin titers.

## Treatment and Prevention
Treatment is available for both immunocompetent and immunocompromised patients. To prevent the disease from recurring, long-term prophylactic treatment with a combination of trimethoprim and sulfamethoxazole may be recommended. The American Academy of Ophthalmology has reported evidence supporting this approach to reduce recurrences.

✅ Research saved to History.

=============== STRICT HYBRID AGENT ===============
1. Research a Topic (Text 💬)
2. Research a Paper (Upload PDF 📖)
3. View History 📜
4. Exit 📤
Select Option: 4
Goodbye.👋
